In [1]:
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from pathlib import Path
from typing import Optional, Tuple
import numpy as np

In [2]:

class CustomDataModule(pl.LightningDataModule):
    def __init__(
        self,
        data_dir: str,
        image_size: Tuple[int, int] = (224, 224),
        batch_size: int = 64,
        val_split: float = 0.2,
        use_augmentation: bool = False,
        num_workers: int = 4,
        seed: int = 42
    ):
        """
        Custom Data Module for handling dataset loading, transformation, and splitting.
        
        Args:
            data_dir (str): Path to dataset directory.
            image_size (Tuple[int, int]): Target image size (height, width).
            batch_size (int): Batch size for DataLoader.
            val_split (float): Fraction of training data to use for validation.
            use_augmentation (bool): Whether to apply data augmentation.
            num_workers (int): Number of workers for DataLoader.
            seed (int): Random seed for reproducibility.
        """
        super().__init__()
        self.data_dir = Path(data_dir)
        self.image_size = image_size
        self.batch_size = batch_size
        self.val_split = val_split
        self.use_augmentation = use_augmentation
        self.num_workers = num_workers
        self.seed = seed
        self.class_names = []
        
        # Define transforms
        self.train_transform = self._get_train_transform()
        self.test_transform = self._get_test_transform()

        # Set manual seeds for reproducibility
        torch.manual_seed(self.seed)
        np.random.seed(self.seed)

    def _get_train_transform(self):
        """Defines transformation pipeline for training data."""
        if self.use_augmentation:
            return transforms.Compose([
                transforms.RandomResizedCrop(self.image_size[0]),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
                transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), shear=10),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        else:
            return transforms.Compose([
                transforms.Resize(self.image_size),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])

    def _get_test_transform(self):
        """Defines transformation pipeline for validation and test data."""
        return transforms.Compose([
            transforms.Resize(self.image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def _split_train_val(self, dataset):
        """Splits dataset into training and validation sets while preserving class distribution."""
        total_size = len(dataset)
        indices = torch.randperm(total_size).tolist()

        val_size = int(total_size * self.val_split)
        train_indices, val_indices = indices[val_size:], indices[:val_size]

        return train_indices, val_indices

    def setup(self, stage: Optional[str] = None):
        """Loads datasets and applies transformations."""
        full_dataset = datasets.ImageFolder(root=self.data_dir, transform=self.train_transform)
        self.class_names = full_dataset.classes

        train_idx, val_idx = self._split_train_val(full_dataset)

        # Create subsets
        self.train_dataset = Subset(full_dataset, train_idx)
        self.val_dataset = Subset(datasets.ImageFolder(root=self.data_dir, transform=self.test_transform), val_idx)

    def train_dataloader(self):
        """Returns DataLoader for training data."""
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)

    def val_dataloader(self):
        """Returns DataLoader for validation data."""
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

    def test_dataloader(self, test_dir: Optional[str] = None):
        """Returns DataLoader for test data."""
        test_path = Path(test_dir) if test_dir else self.data_dir.parent / "val"
        test_dataset = datasets.ImageFolder(root=test_path, transform=self.test_transform)
        return DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)





In [3]:
# Example Usage
# if __name__ == "__main__":
data_module = CustomDataModule(data_dir="/Users/indramandal/Documents/VS_CODE/DA6401/DA6401_Assignment_2/inaturalist_12K/train",
                                use_augmentation=True,
                                val_split= 0.2,
                                batch_size= 64
                                )
data_module.setup()

train_loader = data_module.train_dataloader()
val_loader = data_module.val_dataloader()
test_loader = data_module.test_dataloader()

print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")
print(f"Class Names: {data_module.class_names}")

Training batches: 125
Validation batches: 32
Test batches: 32
Class Names: ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
import torch.nn.functional as F
from torchvision import models
from typing import Tuple


class CustomCNN(pl.LightningModule):
    def __init__(
        self,
        input_shape: Tuple[int, int, int] = (3, 224, 224),
        num_classes: int = 10,
        first_layer_filters: int = 32,
        filter_org: float = 2.0,
        kernel_size: int = 3,
        conv_layers: int = 4,
        activation: str = "relu",
        dropout: float = 0.3,
        batch_norm: bool = True,
        dense_size: int = 128,
        learning_rate: float = 1e-3,
        optimizer_name: str = "adam"
    ):
        """
        Custom CNN Model

        Args:
            input_shape (Tuple[int, int, int]): (Channels, Height, Width)
            num_classes (int): Number of output classes
            first_layer_filters (int): Number of filters in the first layer
            filter_org (float): Scaling factor for filters per layer
            kernel_size (int): Size of the convolution kernel
            conv_layers (int): Number of convolutional layers
            activation (str): Activation function to use (relu, gelu, mish, silu)
            dropout (float): Dropout rate
            batch_norm (bool): Whether to use batch normalization
            dense_size (int): Number of neurons in the fully connected layer
            learning_rate (float): Learning rate
            optimizer_name (str): Optimizer type (adam, sgd, etc.)
        """
        super().__init__()
        self.save_hyperparameters()

        # Choose activation function
        activations = {
            "relu": nn.ReLU(),
            "gelu": nn.GELU(),
            "silu": nn.SiLU(),
            "mish": nn.Mish()
        }
        self.activation = activations.get(activation, nn.ReLU())

        # CNN Layers
        layers = []
        in_channels = input_shape[0]
        filters = first_layer_filters

        for _ in range(conv_layers):
            layers.append(nn.Conv2d(in_channels, filters, kernel_size, padding=1))
            if batch_norm:
                layers.append(nn.BatchNorm2d(filters))
            layers.append(self.activation)
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            in_channels = filters
            filters = int(filters * filter_org)  # Ensure filters remain an integer

        self.conv_block = nn.Sequential(*layers)

        # Calculate output feature map size
        feature_map_size = input_shape[1] // (2 ** conv_layers)  # Assuming max pooling halves the size each time
        if feature_map_size <= 0:
            raise ValueError("Too many pooling layers, feature map size is zero!")

        final_filters = in_channels  # Last number of filters used in conv layers

        # Fully Connected Layers
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(in_features=int(final_filters * feature_map_size * feature_map_size), out_features=dense_size)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(dense_size, num_classes)

        

    def forward(self, x):
        """Defines the forward pass."""
        x = self.conv_block(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

    def training_step(self, batch, batch_idx):
        """Training step for Lightning."""
        images, labels = batch

        # Convert one-hot to class index if needed
        labels = labels.argmax(dim=1) if labels.ndim == 2 else labels  

        preds = self(images)
        loss = F.cross_entropy(preds, labels)
        acc = (preds.argmax(dim=1) == labels).float().mean()
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        """Validation step for Lightning."""
        images, labels = batch

        # Convert one-hot to class index if needed
        labels = labels.argmax(dim=1) if labels.ndim == 2 else labels  

        preds = self(images)
        loss = F.cross_entropy(preds, labels)
        acc = (preds.argmax(dim=1) == labels).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        """Test step for Lightning."""
        images, labels = batch

        # Convert one-hot to class index if needed
        labels = labels.argmax(dim=1) if labels.ndim == 2 else labels  

        preds = self(images)
        loss = F.cross_entropy(preds, labels)
        acc = (preds.argmax(dim=1) == labels).float().mean()
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", acc, prog_bar=True)
        return loss


    def configure_optimizers(self):
        """Optimizer & Scheduler setup."""
        optimizers = {"adam": optim.Adam, "sgd": optim.SGD}
        optimizer = optimizers[self.hparams.optimizer_name](self.parameters(), lr=self.hparams.learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
        return [optimizer], [scheduler]


In [5]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from torchsummary import summary

# Instantiate DataModule
data_module = CustomDataModule(data_dir="/Users/indramandal/Documents/VS_CODE/DA6401/DA6401_Assignment_2/inaturalist_12K/train",
                               use_augmentation=True,
                               val_split=0.25,
                               batch_size=64)

# Call setup() to initialize datasets
data_module.setup(stage="fit")

# Instantiate Model with correct num_classes
model = CustomCNN(num_classes=len(data_module.class_names),
                  first_layer_filters=64,
                  filter_org=1.0,
                  kernel_size=3,
                  conv_layers=5,
                  activation="relu",
                  dropout=0.2,
                  batch_norm=True,
                  dense_size=128,
                  learning_rate=1e-3,
                  optimizer_name="adam")

# Print Detailed Model Summary with torchsummary
print("Detailed Model Summary:")
summary(model, input_size=(3, 224, 224))  # Adjust the input size depending on your model

# Callbacks for Training
early_stop = EarlyStopping(monitor="val_loss", patience=5, mode="min")
checkpoint = ModelCheckpoint(dirpath="checkpoints/", save_top_k=1, monitor="val_loss", mode="min")

# Setup TensorBoardLogger for better logging and visualization
logger = TensorBoardLogger("tb_logs", name="my_model")

# Train the Model
trainer = Trainer(
    max_epochs=5,
    precision=16,  # Mixed precision for speed
    callbacks=[early_stop, checkpoint],
    logger=logger
)

trainer.fit(model, datamodule=data_module)

# Test Model
trainer.test(model, datamodule=data_module)



ModuleNotFoundError: No module named 'torchsummary'